**Instalación de dependencias**

Agrega los paquetes necesarios a tu proyecto:

In [ ]:
%pip install azure-ai-inference[opentelemetry]
%pip install azure-search-documents 
%pip install azure-identity
%pip install openai

**Configuración de la aplicación**

Aquí está la configuración de tu aplicación, podrías mejorarla usando un archivo .env en lugar de dejar aquí todas tus variables!
¿Puedes con eso?

In [ ]:
import os
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import dotenv
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

# Leer las variables definidas en el archivo .env

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME = "text-embedding-ada-002"

AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX")
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")

openai_client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_version=OPENAI_API_VERSION
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX,
    credential=AzureKeyCredential(AZURE_SEARCH_ADMIN_KEY)
)

def get_embedding(text):
    return openai_client.embeddings.create(
        model=AZURE_OPENAI_EMBEDDING_DEPLOYED_MODEL_NAME,
        input=text
    ).data[0].embedding

**Haz la pregunta**

Pregúntale al RAG

In [3]:
user_question = "¿En qué condiciones climáticas se recomienda sembrar aguacate?"
user_question_vector = get_embedding(user_question)
print(user_question_vector)

[-0.009611524641513824, -0.013852473348379135, 0.022954050451517105, -0.003992429934442043, 0.007068247068673372, 0.005764333065599203, -0.011502845212817192, 0.01718326285481453, -0.010741153731942177, 0.004005339927971363, 0.026852883398532867, 0.002565870527178049, 0.0037084089126437902, 0.0022253680508583784, 0.0016686227172613144, 0.006325919646769762, 0.020927174016833305, -0.0010255413362756371, 0.026878703385591507, -0.010515227913856506, -0.003766504116356373, 0.017751304432749748, 0.004495921544730663, -0.01757056452333927, -0.024567805230617523, -0.0028966902755200863, -0.0010650783078745008, -0.028014786541461945, 0.003038700670003891, -0.013258610852062702, 0.03785223886370659, -0.006416290067136288, -0.02273458056151867, 0.006451792549341917, -0.013981573283672333, -0.026852883398532867, -0.012709934264421463, -0.01615045964717865, 0.010754063725471497, -0.007261897437274456, 0.015130567364394665, 0.008959568105638027, 0.009605069644749165, -0.00034272120683453977, 0.0164

**Muestra la pregunta en un modo que se puede leer**

Convierte los resultados de tu RAG en un formato que se pueda leer.

In [ ]:
search_results = search_client.search(
    search_text=None,
    top=3,
    vector_queries=[
        VectorizedQuery(
            vector=get_embedding(user_question),
            k_nearest_neighbors=3,
            fields="text_vector"
        )
    ]
)

for result in search_results:
    print("Chunk ID:", result["chunk_id"])
    print("Title:", result["title"])
    print("Text:", result["chunk"])
    print()

Chunk ID: 7db6dcece49d_aHR0cHM6Ly9zdGd0b3BpY29zcmFnLmJsb2IuY29yZS53aW5kb3dzLm5ldC9yYWctZG9jcy9ESU1fTWFudWFsX2RlX2N1bHRpdm9fZGVfaG9ydGFsaXphcy5wZGY1_pages_11
Title: DIM_Manual_de_cultivo_de_hortalizas.pdf
Text: y rala dejando caer la 
semilla en el fondo de un pequeño 
surco a 1 o 5 centímetros de 
profundidad. Posteriormente se ralea dejando las plantas a la distancia 



Manual para el Productor: El Cultivo de las Hortalizas

Proyecto: Manejo Integral de los Recursos Naturales en el Trópico de Cochabamba y Los Yungas de La Paz

16

adecuada. Ejemplo: rábano, 
zanahoria, espinaca. En este 
sistema los cultivos de ciclo tardío 
se siembran en surcos espaciados 
a un metre entre ello y de 45 a 70 
centímetros para cultivos de ciclo 
precoz.

c) A golpes.- Se siembra la semilla en 
pequeños huecos distanciados, 
colocando 2 a 3 semillas en cada 
hueco.

4.1.2 DIMENSIONADO

Se realiza bajo el método de tres bolillos y cuadrado. En este sistema 
se hacen agujeros de 2 a 5 centímetros de pro

**Agrega un LLM a tu RAG**

Muestra cómo puedes agregar un modelo de lenguaje a tu RAG para mejorar la calidad de las respuestas.

In [ ]:
AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME = "gpt-4o-mini"

context = ""
for result in search_results:
    context += result["chunk"] + "\n\n"

SYSTEM_MESSAGE = f"""
You are an AI Assistant.
Be brief in your answers. Answer ONLY with the facts listed in the retrieved text.

Context:
{context}
"""

USER_MESSAGE = user_question

response = openai_client.chat.completions.create(
    model=AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME,
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

El aguacate se recomienda sembrar en climas cálidos y húmedos, con temperaturas entre 20°C y 30°C.
